In [41]:
from pprint import pprint
from pathlib import Path
import re
import pandas as pd

# with open('LTFiles/all_inductors.lib') as inductors:
#     re.findall(r"(?s)(?=\*)(.*?)(?=\.subckt)(.*?)(?<=\.ends)(.*?)(?=\n)", inductors.rea, flags=re.S)

# with open('LTFiles/subset_inductors.lib') as inductors:

def process_coilcraft_inductor_lib(filename: str):
    data_str = Path(filename).read_text()
    data = re.findall(r"(?s)(?=\*)(.*?)(?=\.subckt)(.*?)(?<=\.ends)(.*?)(?=\n)", data_str, flags=re.S)  # This expression extracts each subckt 'clump'
    data = [''.join(x) for x in data]  # This takes all the groups that are generated in the above step and merges them into one string

    data = [x for x in data if '_freq' not in x]  # Remove any of the frequency simulations since those are for frequency domain analysis and we are doing time

    # pprint(data)
    ret_data = []
    inductance_regex = r"(?<=Inductance = )(.*?)(?=\n)"
    part_number_regex = r"(?<=Coilcraft Part Number : )(.*?)(?=\n)"
    for inductor in data:
        try:
            ret_data.append({"PartNumber": re.findall(part_number_regex, inductor)[0], "Inductance": re.findall(inductance_regex, inductor)[0], "SubCkt": inductor.replace('\n', ' ')})
        except:
            continue

    return ret_data

coilcraft_inductors = process_coilcraft_inductor_lib('LTFiles/all_inductors.lib')
pprint(coilcraft_inductors)
df = pd.DataFrame(coilcraft_inductors)
df.to_csv('ProcessedInductors.csv')


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [49]:
import pandas as pd

models = pd.read_csv('ProcessedInductors.csv')

for index, item in models.iterrows():
    if item.PartNumber == 'MODEL':
        print(item)

Unnamed: 0                                                   27
PartNumber                                                MODEL
Inductance                                                    0
SubCkt        *=============================================...
Name: 27, dtype: object
Unnamed: 0                                                   28
PartNumber                                                MODEL
Inductance                                                    0
SubCkt        *=============================================...
Name: 28, dtype: object
Unnamed: 0                                                   38
PartNumber                                                MODEL
Inductance                                                    0
SubCkt        *=============================================...
Name: 37, dtype: object
Unnamed: 0                                                  842
PartNumber                                                MODEL
Inductance                      

In [73]:
from LTTraceData import LTTraceData
from PyLTSpice import SimCommander, RawRead

high_side = True
low_side = False

# select spice model
LTC = SimCommander("LTFiles/InSchematicSubCktExample.asc")
# set default arguments
print(LTC.get_components())
# print(LTC.get_component_info('X1'))

Using LTspice installed in : 'C:\Users\treys\AppData\Local\Programs\ADI\LTspice\LTspice.exe' 
Creating Netlist
The Netlist was successfully created
['R1', 'V1', 'X1']


In [11]:
# import PyLTSpice.raw_read
from LTTraceData import LTTraceData
from PyLTSpice import SimCommander, RawRead

high_side = True
low_side = False

def frequencytoltpulse(frequency, side: bool):
    deadband = 8  # ns
    period = 1 / (frequency / 1E9)  # Convert to Gigahertz to ensure the period will be in nanoseconds
    if side == low_side:
        return f"PULSE(3.3 0 0n 1n 1n {period + (deadband * 2)}n {period * 2}n)"
    else:
        return f"PULSE(0 3.3 {deadband}n 1n 1n {period}n {period * 2}n)"

fsw = 100E3
L = 10E-6
C = 47E-6

# select spice model
# LTC = SimCommander("LTFiles/EPC23102_Mine.asc")
# set default arguments
# LTC.set_component_value('VHSin', frequencytoltpulse(fsw, high_side))
# LTC.set_component_value('VLSin', frequencytoltpulse(fsw, low_side))

In [15]:
run_netlist_file = f'LTFiles/F{fsw}_L{L}_C{C}'
LTC.run(run_filename=f'{run_netlist_file}.net')

Wed Apr 26 12:04:15 2023 : Starting simulation 1


<RunTask(sim1, started 7876)>

In [16]:
LTR = RawRead(f'{run_netlist_file}.raw')
print(LTR.get_trace_names())

v_in = LTTraceData(LTR, 'V(vin)')
v_out = LTTraceData(LTR, 'V(vout)')
i_in = LTTraceData(LTR, 'I(Vs)')
i_out = LTTraceData(LTR, 'I(Il)')

LTC.set_element_model()

print(v_out.average)
print(i_out.average)

print(v_in.average)
print(i_in.average)

print(f'Eff: {abs(v_out.average * i_out.average) / abs(v_in.average * i_in.average) * 100}')

Reading file with encoding  utf_16_le
File contains 8 traces, reading 8
Binary RAW file with Normal access
['time', 'V(vout)', 'V(sw)', 'V(vin)', 'I(C1)', 'I(L1)', 'I(Il)', 'I(Vs)']
5.005125
1.0
10.0
-0.20583676
Eff: 243.15993785858154


In [5]:
import numpy as np
import pandas as pd

inductors = pd.read_csv('Inductors.csv')

# with open("Inductors.csv", 'r', encoding='utf-8-sig') as x:  # https://stackoverflow.com/a/49150749
#     arr = np.array(list(csv.reader(x, delimiter=",")))
#
#     inductances = []
#     for elem in arr:
#         inductances.append(elem[1])
#
#     print(inductances)

In [39]:
def find_nearest(array, value):  # Thanks: https://stackoverflow.com/a/2566508
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

lowest_dcr_inductors = inductors.groupby('Inductance_uH', group_keys=False).apply(lambda x: x.loc[x.DCR_mOhm.idxmin()])  # https://stackoverflow.com/a/13059751

required_inductor = find_nearest(lowest_dcr_inductors.Inductance_uH, 9.8)
print(required_inductor)
required_inductor_row = lowest_dcr_inductors.loc[lowest_dcr_inductors.Inductance_uH == required_inductor]  # https://stackoverflow.com/a/17071908
print(required_inductor_row.DCR_mOhm)

10.0
Inductance_uH
10.0    6.3
Name: DCR_mOhm, dtype: float64
